In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
import warnings

warnings.filterwarnings('ignore')
sns.set_theme(style = 'darkgrid',palette = 'dark')

In [3]:
data1 = pd.read_csv('2016_Building_Energy_Benchmarking.csv')

In [4]:
data1

,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,Address,City,State,ZipCode,TaxParcelIdentificationNumber,...,Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),DefaultData,Comments,ComplianceStatus,Outlier,TotalGHGEmissions,GHGEmissionsIntensity
0,1,2016,NonResidential,Hotel,Mayflower park hotel,405 Olive way,Seattle,WA,98101.0,0659000030,...,1.156514e+06,3.946027e+06,12764.529300,1.276453e+06,False,NaN,Compliant,NaN,249.98,2.83
1,2,2016,NonResidential,Hotel,Paramount Hotel,724 Pine street,Seattle,WA,98101.0,0659000220,...,9.504252e+05,3.242851e+06,51450.816410,5.145082e+06,False,NaN,Compliant,NaN,295.86,2.86
2,3,2016,NonResidential,Hotel,5673-The Westin Seattle,1900 5th Avenue,Seattle,WA,98101.0,0659000475,...,1.451544e+07,4.952666e+07,14938.000000,1.493800e+06,False,NaN,Compliant,NaN,2089.28,2.19
3,5,2016,NonResidential,Hotel,HOTEL MAX,620 STEWART ST,Seattle,WA,98101.0,0659000640,...,8.115253e+05,2.768924e+06,18112.130860,1.811213e+06,False,NaN,Compliant,NaN,286.43,4.67
4,8,2016,NonResidential,Hotel,WARWICK SEATTLE HOTEL (ID8),401 LENORA ST,Seattle,WA,98121.0,0659000970,...,1.573449e+06,5.368607e+06,88039.984380,8.803998e+06,False,NaN,Compliant,NaN,505.01,2.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3371,50222,2016,Nonresidential COS,Office,Horticulture building,1600 S Dakota St,Seattle,WA,NaN,1624049080,...,1.536550e+05,5.242709e+05,3254.750244,3.254750e+05,True,NaN,Error - Correct Default Data,NaN,20.94,1.70
3372,50223,2016,Nonresidential COS,Other,International district/Chinatown CC,719 8th Ave S,Seattle,WA,NaN,3558300000,...,1.162210e+05,3.965461e+05,5537.299805,5.537300e+05,False,NaN,Compliant,NaN,32.17,2.01
3373,50224,2016,Nonresidential COS,Other,Queen Anne Pool,1920 1st Ave W,Seattle,WA,NaN,1794501150,...,5.252517e+05,1.792159e+06,39737.390630,3.973739e+06,False,NaN,Compliant,NaN,223.54,16.99
3374,50225,2016,Nonresidential COS,Mixed Use Property,South Park Community Center,8319 8th Ave S,Seattle,WA,NaN,7883603155,...,1.022480e+05,3.488702e+05,3706.010010,3.706010e+05,False,NaN,Compliant,NaN,22.11,1.57


In [5]:
data1.describe()

,OSEBuildingID,DataYear,ZipCode,CouncilDistrictCode,Latitude,Longitude,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,...,SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),Comments,TotalGHGEmissions,GHGEmissionsIntensity
count,3376.000000,3376.0,3360.000000,3376.000000,3376.000000,3376.000000,3376.000000,3368.000000,3376.000000,3.376000e+03,...,3.371000e+03,3.370000e+03,3.367000e+03,3.367000e+03,3.367000e+03,3.367000e+03,3.367000e+03,0.0,3367.000000,3367.000000
mean,21208.991114,2016.0,98116.949107,4.439277,47.624033,-122.334795,1968.573164,1.106888,4.709123,9.483354e+04,...,5.403667e+06,5.276726e+06,2.745959e+05,1.086639e+06,3.707612e+06,1.368505e+04,1.368505e+06,NaN,119.723971,1.175916
std,12223.757015,0.0,18.615205,2.120625,0.047758,0.027203,33.088156,2.108402,5.494465,2.188376e+05,...,2.161063e+07,1.593879e+07,3.912173e+06,4.352478e+06,1.485066e+07,6.709781e+04,6.709781e+06,NaN,538.832227,1.821452
min,1.000000,2016.0,98006.000000,1.000000,47.499170,-122.414250,1900.000000,0.000000,0.000000,1.128500e+04,...,0.000000e+00,0.000000e+00,0.000000e+00,-3.382680e+04,-1.154170e+05,0.000000e+00,0.000000e+00,NaN,-0.800000,-0.020000
25%,19990.750000,2016.0,98105.000000,3.000000,47.599860,-122.350662,1948.000000,1.000000,2.000000,2.848700e+04,...,9.251286e+05,9.701822e+05,0.000000e+00,1.874229e+05,6.394870e+05,0.000000e+00,0.000000e+00,NaN,9.495000,0.210000
50%,23112.000000,2016.0,98115.000000,4.000000,47.618675,-122.332495,1975.000000,1.000000,4.000000,4.417500e+04,...,1.803753e+06,1.904452e+06,0.000000e+00,3.451299e+05,1.177583e+06,3.237538e+03,3.237540e+05,NaN,33.920000,0.610000
75%,25994.250000,2016.0,98122.000000,7.000000,47.657115,-122.319407,1997.000000,1.000000,5.000000,9.099200e+04,...,4.222455e+06,4.381429e+06,0.000000e+00,8.293178e+05,2.829632e+06,1.189033e+04,1.189034e+06,NaN,93.940000,1.370000
max,50226.000000,2016.0,98272.000000,7.000000,47.733870,-122.220966,2015.000000,111.000000,99.000000,9.320156e+06,...,8.739237e+08,4.716139e+08,1.349435e+08,1.925775e+08,6.570744e+08,2.979090e+06,2.979090e+08,NaN,16870.980000,34.090000


In [6]:
data1.duplicated().sum()

0

In [7]:
liste_non_residentiel = ['NonResidential',
                         'Nonresidential COS',
                         'SPS-District K-12',
                         'Campus',
                         'Nonresidential WA']

In [8]:
data = data1.loc[data1['BuildingType'].isin(liste_non_residentiel)]
data.shape

(1668, 46)

In [9]:
data.select_dtypes(include=['float', 'int']).columns

Index(['OSEBuildingID', 'DataYear', 'ZipCode', 'CouncilDistrictCode',
       'Latitude', 'Longitude', 'YearBuilt', 'NumberofBuildings',
       'NumberofFloors', 'PropertyGFATotal', 'PropertyGFAParking',
       'PropertyGFABuilding(s)', 'LargestPropertyUseTypeGFA',
       'SecondLargestPropertyUseTypeGFA', 'ThirdLargestPropertyUseTypeGFA',
       'ENERGYSTARScore', 'SiteEUI(kBtu/sf)', 'SiteEUIWN(kBtu/sf)',
       'SourceEUI(kBtu/sf)', 'SourceEUIWN(kBtu/sf)', 'SiteEnergyUse(kBtu)',
       'SiteEnergyUseWN(kBtu)', 'SteamUse(kBtu)', 'Electricity(kWh)',
       'Electricity(kBtu)', 'NaturalGas(therms)', 'NaturalGas(kBtu)',
       'Comments', 'TotalGHGEmissions', 'GHGEmissionsIntensity'],
      dtype='object')

In [10]:
data['NumberofBuildings'].unique()

array([  1.,   3.,   0.,   2.,   4.,  27.,   6.,  11.,  14.,   9.,   5.,
        nan,   7.,   8.,  23.,  10., 111.])

In [11]:
# j'aurais pu vérifier un par un le nombre de batiment, mais comme il n'y a pas de campus, je vais supposer que c'est 1.
data.loc[(data['NumberofBuildings'].isnull()) | (data['NumberofBuildings'] == 0) ]


,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,Address,City,State,ZipCode,TaxParcelIdentificationNumber,...,Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),DefaultData,Comments,ComplianceStatus,Outlier,TotalGHGEmissions,GHGEmissionsIntensity
63,89,2016,NonResidential,Medical Office,Polyclinic North && South,1200 Harvard Ave,Seattle,WA,98122.0,1978201270,...,2.897563e+06,9886484.0,68145.703130,6814570.0,False,NaN,Compliant,NaN,430.84,2.41
76,113,2016,NonResidential,Small- and Mid-Sized Office,KCTS9,401 Mercer St,Seattle,WA,98109.0,1988200705,...,1.221074e+06,4166305.0,0.000000,0.0,False,NaN,Compliant,NaN,29.04,0.44
160,257,2016,NonResidential,Warehouse,Harbor Island: Warehouse E - DNRP-SWD,3235 16TH AVE SW,Seattle,WA,98134.0,7666703020,...,7.067415e+05,2411402.0,0.000000,0.0,False,NaN,Compliant,NaN,16.81,0.21
201,322,2016,NonResidential,Large Office,Cox Media Group Seattle KIRO TV (322),2801 3rd Ave,Seattle,WA,98121.0,0656000005,...,2.868908e+06,9788716.0,0.000000,0.0,False,NaN,Compliant,NaN,68.24,0.68
241,366,2016,NonResidential,Small- and Mid-Sized Office,University District Office Buiding,1107 NE 45th St,Seattle,WA,98105.0,1142000500,...,8.391593e+05,2863212.0,18652.765630,1865277.0,False,NaN,Compliant,NaN,119.03,1.20
264,391,2016,NonResidential,Mixed Use Property,Thaw Building,8300 Military Rd South,Seattle,WA,98108.0,0007400015,...,7.416934e+05,2530658.0,18524.421880,1852442.0,False,NaN,Compliant,NaN,116.03,1.31
278,408,2016,NonResidential,Large Office,Fourth and Pike Building,1424 4th Ave,Seattle,WA,98101.0,1975700235,...,1.670053e+06,5698221.0,25.510000,2551.0,False,NaN,Compliant,NaN,65.20,0.49
280,410,2016,NonResidential,Hotel,Motif Seattle,1414 4th Ave,Seattle,WA,98101.0,1975700255,...,5.803207e+06,19800542.0,23989.949220,2398995.0,False,NaN,Compliant,NaN,1468.50,4.42
291,421,2016,NonResidential,Other,Meridian (East),1501 7th Ave,Seattle,WA,98101.0,1976700055,...,1.515212e+06,5169902.0,21032.531250,2103253.0,False,NaN,Compliant,NaN,147.75,0.55
358,504,2016,Nonresidential COS,Other,Westbridge,4209 W Marginal Way SW,Seattle,WA,98118.0,1824049020,...,4.310917e+05,1470885.0,21036.570310,2103657.0,False,NaN,Compliant,NaN,121.98,1.07


In [12]:
data.loc[(data['NumberofBuildings'].isnull()) | (data['NumberofBuildings'] == 0), 'NumberofBuildings' ] = 1

In [13]:
# j'ai vérifié si les buildings respectaient bien ce qu'elle doivent être pour les grandes valeurs potentiellement aberrantes.
data.loc[data['NumberofBuildings'] >= 20,['PropertyName','NumberofBuildings']]

,PropertyName,NumberofBuildings
105,SSCC MAIN CAMPUS,27.0
1862,FT C15 Fishermen's Center,23.0
3274,University of Washington - Seattle Campus,111.0


In [14]:
data['NumberofFloors'].unique()

array([12, 11, 41, 10, 18,  2,  8, 15, 25,  9, 33,  6, 28,  5, 19,  7,  1,
        3,  4, 24, 20, 34,  0, 16, 23, 17, 36, 22, 47, 29, 14, 49, 37, 42,
       63, 13, 21, 55, 46, 30, 56, 76, 27, 99, 39], dtype=int64)

In [15]:
# Valeurs erronée après recherche google maps : l'église (index = 1359) aux 99 étages (a changer en 2 floors), le reste est bon
data.loc[data['NumberofFloors'] >=50, ['PropertyName','NumberofFloors']]

,PropertyName,NumberofFloors
233,Seattle Municipal Tower (2030),63
271,1201 Third Avenue,55
292,Two Union Square,56
559,Columbia Center - 2015,76
1359,Seattle Chinese Baptist Church,99


In [16]:
data.loc[data['NumberofFloors'] == 99, "NumberofFloors"] = 2

In [17]:
# on regarde maintenant les valeurs égales à 0 (aberrantes) 
data.loc[data['NumberofFloors'] == 0, ['PropertyName' , 'Address' , 'BuildingType' , 'NumberofFloors']]

,PropertyName,Address,BuildingType,NumberofFloors
166,Grand Hyatt Seattle,721 Pine St,NonResidential,0
487,Arnold Pavilion,1221 Madison Street,NonResidential,0
488,2200 Westlake - SEDO,2200 Westlake Ave.,NonResidential,0
564,Pacific Place,600 Pine Street,NonResidential,0
1754,HART First Hill LLC,1124 Columbia Street,NonResidential,0
1993,(ID#24086)Campus1:KC Metro Transit Atlantic Ce...,1333 AIRPORT WAY S,Campus,0
3130,Sandpoint #5,7561 63rd Ave NE,NonResidential,0
3131,Sandpoint #25,6222 NE 74th St,NonResidential,0
3132,Sandpoint #29,6200 NE 74th ST,NonResidential,0
3168,Magnuson,7400 Sand Point Way NE,Nonresidential COS,0


In [18]:
# Je m'amuse sur maps a créer la liste du nombre d'étage dans l'ordre :
liste_etage = [2, 12, 4, 7, 1, 1, 3, 3, 2, 2, 3, 6, 6, 8, 1, 1]
data.loc[data['NumberofFloors'] == 0, 'NumberofFloors'] = liste_etage

In [19]:
# Regardons les aberration dans les energies maintenant. Des valeurs d'electricité négative (peut être la produise-t-il?)
# Ou des valeurs totale d'energies à 0, mais pas les valeurs indépendament prise sur electricity/steam/gaz.
data.loc[data['SiteEnergyUse(kBtu)']<=0, ['PropertyName','BuildingType','SiteEnergyUse(kBtu)','Electricity(kBtu)','SteamUse(kBtu)','NaturalGas(kBtu)']]

,PropertyName,BuildingType,SiteEnergyUse(kBtu),Electricity(kBtu),SteamUse(kBtu),NaturalGas(kBtu)
28,Meany Building,SPS-District K-12,0.0,0.0,0.0,0.0
31,John Hay Elementary,SPS-District K-12,0.0,1496816.0,0.0,0.0
62,Arbor Heights Elementary,SPS-District K-12,0.0,600813.0,0.0,0.0
81,Pathfinder K-8,SPS-District K-12,0.0,1698176.0,0.0,0.0
85,John Muir Elementary,SPS-District K-12,0.0,2347138.0,0.0,0.0
95,B.F. Day Elementary,SPS-District K-12,0.0,2104286.0,0.0,0.0
133,Whitman Middle,SPS-District K-12,0.0,0.0,0.0,4318917.0
139,Washington Middle,SPS-District K-12,0.0,0.0,0.0,3217877.0
152,Olympic View Elementary,SPS-District K-12,0.0,0.0,0.0,0.0
304,Washington State Convention Center,NonResidential,0.0,0.0,0.0,0.0


In [20]:
# On additionne tout simplement les energies là ou elles sont non nulles mais que leur total l'est.
data.loc[data['SiteEnergyUse(kBtu)']<=0,
         'SiteEnergyUse(kBtu)'
        ] = data['Electricity(kBtu)'] + data['SteamUse(kBtu)'] + data['NaturalGas(kBtu)']

In [21]:
# Il reste des valeur toutes à 0 pour les energies. On les np.nan et on leur appliquera un Knn.
data.loc[data['SiteEnergyUse(kBtu)']<=0, 
         ['PropertyName','BuildingType','SiteEnergyUse(kBtu)','Electricity(kBtu)','SteamUse(kBtu)','NaturalGas(kBtu)']
        ]

,PropertyName,BuildingType,SiteEnergyUse(kBtu),Electricity(kBtu),SteamUse(kBtu),NaturalGas(kBtu)
28,Meany Building,SPS-District K-12,0.0,0.0,0.0,0.0
152,Olympic View Elementary,SPS-District K-12,0.0,0.0,0.0,0.0
304,Washington State Convention Center,NonResidential,0.0,0.0,0.0,0.0
746,Welcome Home Society - PriceCo,NonResidential,0.0,0.0,0.0,0.0
1361,Olympic Hills Elementary,SPS-District K-12,0.0,0.0,0.0,0.0


In [22]:
List_a_nan = ['Electricity(kBtu)','SteamUse(kBtu)','NaturalGas(kBtu)','SiteEnergyUse(kBtu)']
for i in List_a_nan :
    data.loc[data['SiteEnergyUse(kBtu)']<=0, i] = np.nan

In [23]:
data.loc[data['Electricity(kBtu)']<=0, ['PropertyName','BuildingType','SiteEnergyUse(kBtu)','Electricity(kBtu)','SteamUse(kBtu)','NaturalGas(kBtu)']]

,PropertyName,BuildingType,SiteEnergyUse(kBtu),Electricity(kBtu),SteamUse(kBtu),NaturalGas(kBtu)
133,Whitman Middle,SPS-District K-12,4.318917e+06,0.0,0.0,4318917.0
139,Washington Middle,SPS-District K-12,3.217877e+06,0.0,0.0,3217877.0
426,Pier 57,NonResidential,1.150804e+07,0.0,0.0,11508035.0
513,IUC- Whole Foods Interbay,NonResidential,1.252517e+07,0.0,0.0,0.0
1894,North Beach Elementary,SPS-District K-12,9.455370e+05,0.0,0.0,945537.0
3166,Catharine Blaine K-8,SPS-District K-12,4.993620e+06,0.0,0.0,4993620.0
3206,Bullitt Center,NonResidential,3.427261e+05,-115417.0,0.0,0.0


In [24]:
# Certains individus ont une energie totale strictement supérieur à 0, mais 0 dans chacune des energies utilisés (ou même des
# valeurs négatives). On va considérer que ces batiments sont à 100% à l'electricité.
data.loc[data['Electricity(kBtu)']<0, 'Electricity(kBtu)'] = data['SiteEnergyUse(kBtu)']
data.loc[(data['SiteEnergyUse(kBtu)']>0) & 
         (data['Electricity(kBtu)']==0) & 
         (data['SteamUse(kBtu)']==0) & 
         (data['NaturalGas(kBtu)']==0),
        'Electricity(kBtu)'
        ] = data['SiteEnergyUse(kBtu)']

In [25]:
data.loc[data['SteamUse(kBtu)']<0, 
         ['PropertyName','BuildingType','SiteEnergyUse(kBtu)','Electricity(kBtu)','SteamUse(kBtu)','NaturalGas(kBtu)']
        ]

,PropertyName,BuildingType,SiteEnergyUse(kBtu),Electricity(kBtu),SteamUse(kBtu),NaturalGas(kBtu)


In [26]:
data.loc[data['NaturalGas(kBtu)']<0, 
         ['PropertyName','BuildingType','SiteEnergyUse(kBtu)','Electricity(kBtu)','SteamUse(kBtu)','NaturalGas(kBtu)']
        ]

,PropertyName,BuildingType,SiteEnergyUse(kBtu),Electricity(kBtu),SteamUse(kBtu),NaturalGas(kBtu)


In [27]:
# On vire les valeur nulle d'emission de gaz (il y en a 5) pour les remplacer par la suite.

data.loc[data['TotalGHGEmissions']<=0, 'TotalGHGEmissions'] = np.nan

In [28]:
data.isna().sum()

OSEBuildingID                         0
DataYear                              0
BuildingType                          0
PrimaryPropertyType                   0
PropertyName                          0
Address                               0
City                                  0
State                                 0
ZipCode                              16
TaxParcelIdentificationNumber         0
CouncilDistrictCode                   0
Neighborhood                          0
Latitude                              0
Longitude                             0
YearBuilt                             0
NumberofBuildings                     0
NumberofFloors                        0
PropertyGFATotal                      0
PropertyGFAParking                    0
PropertyGFABuilding(s)                0
ListOfAllPropertyUseTypes             2
LargestPropertyUseType                6
LargestPropertyUseTypeGFA             6
SecondLargestPropertyUseType        813
SecondLargestPropertyUseTypeGFA     813


In [29]:
# On fait le Knn. Il y a peu de valeurs a fill, j'impose un 3nn sans spécialement d'étude préalables.
columns_a_fillna = ['ENERGYSTARScore', 
                    'SiteEUI(kBtu/sf)', 
                    'SiteEUIWN(kBtu/sf)',
                    'SourceEUI(kBtu/sf)', 
                    'SourceEUIWN(kBtu/sf)', 
                    'SiteEnergyUse(kBtu)',
                    'SiteEnergyUseWN(kBtu)',
                    'SteamUse(kBtu)', 
                    'Electricity(kWh)',
                    'Electricity(kBtu)', 
                    'NaturalGas(therms)', 
                    'NaturalGas(kBtu)',
                    'TotalGHGEmissions']

In [30]:
x = data[columns_a_fillna]
scaler = StandardScaler()
scaler.fit(x)
X_cr = scaler.transform(x)
imputer = KNNImputer(n_neighbors=3)
X_knn = imputer.fit_transform(X_cr)
data[columns_a_fillna] = scaler.inverse_transform(X_knn)

In [31]:
data.loc[data['SiteEnergyUse(kBtu)']<=0, 
         ['PropertyName','BuildingType','SiteEnergyUse(kBtu)','Electricity(kBtu)','SteamUse(kBtu)','NaturalGas(kBtu)']
        ]

,PropertyName,BuildingType,SiteEnergyUse(kBtu),Electricity(kBtu),SteamUse(kBtu),NaturalGas(kBtu)


In [32]:
# On va maintenant créer de nouvelles colonnes utiles. 

# L'énnoncé du problème indique que l'on aura le renseignement de la proportion d'energie utilisé, sans pour autant avoir accès
# aux energies elles même. Ajoutons donc ces informations dans notre dataset.
data["proportion steam (en %)"] = round(data['Electricity(kBtu)']*100/data['SiteEnergyUse(kBtu)'],2)
data["proportion electricity (en %)"] = round(data['SteamUse(kBtu)']*100/data['SiteEnergyUse(kBtu)'],2)
data["proportion gas (en %)"] = round(data['NaturalGas(kBtu)']*100/data['SiteEnergyUse(kBtu)'],2)
data["autres energies (en %)"] = round(100 - data["proportion gas (en %)"] - data["proportion electricity (en %)"] - data["proportion steam (en %)"],2)

# On créer maintenant des colonnes supplémentaires indiquant ou le batiment se trouve :
dummies = pd.get_dummies(data['Neighborhood'], columns = ['Neighborhood'])
data = data.join(dummies)


In [33]:
# Nous normaliseons au besoin lors des études des différents modèles prédictif. Nous avons donc un dataset prêt pour
# l'étude prédictive, nous l'enregistrons donc.
data.to_csv('data_final.csv')